# Supply chain problem
This project was part of a consulting project led at the Graduate School of Management at the University of Saint Petersburg (SPbU).

Problem: in the post sanction era, the client faced lack of shipping containers and European trucks, difficulty reaching global markets due to restrictions, and unbalanced mill operations.

Request: calculate the best transportation allocation post-sanctions, based on 2021 data.

Data: list of deliveries for each customer.

## Accomplishments
In this notebook, I have successfully accomplished the following tasks:

✅ Data engineering: standardize features, such as cost, to a single type of currency at a historical rate

✅ Data engineering : mill capacity and mill - delivery-point distance calculation using forward geocoding for data collection and using Haversine method for calculation

✅ Data analysis : assessing and forecasting performance of several mills under specific circumstances (such as sanctions)

✅ Data visualisation : using the abovementionned methods to plot datapoints and using scales to identify strong demand and/or trends

## Steps to accomplish
1) To sort out which deliveries are still relevant post sanctions. All EU28 countries as well as a handful of non-EU countries will impact the demand.
2) To calculate the new allocation of transport per mill depending on the maximum capacity and cost per deliveries.

## Notes
The data was anonymised for the purpose of this portfolio. Any data related to identifiable sources, such as names, have been transformed. Since the data contains geographic information, some visualisations are hidden. For any questions or enquiries, please contact me by email : celine.argyropoulos@gmail.com

In [1]:
import pandas as pd

In [2]:
# Checking the data
df = pd.read_excel(r'./data/transportation_data.xlsx')
print(df.describe())
df.head()

                cost      order_id
count   12967.000000  1.296700e+04
mean    15215.164494  8.769522e+06
std     24621.476914  3.282550e+06
min       375.000000  9.552140e+05
25%      1900.000000  1.008156e+07
50%      2550.000000  1.013192e+07
75%     17000.000000  1.018170e+07
max    175000.000000  1.023331e+07


,mill,country_delivery,country,delivery_point,cost,order_id,month_order
0,Hobart,AU,Australia,Nowra,175000,10226209,November
1,Hobart,AU,Australia,Nowra,175000,10226230,November
2,Darwin,AU,Australia,Newcastle,171000,10206123,October
3,Darwin,AU,Australia,Newcastle,171000,10185874,September
4,Darwin,AU,Australia,North-west,170000,10176556,August


In [3]:
# check missing values
print(df.dtypes)
print("Missing values: ",'\n',df.isnull().sum())
# check unique values
order_id_missing = df['order_id'].isnull().sum()
order_id_unique = df['order_id'].is_unique
order_id_type = df['order_id'].dtype
print(df.columns[0],order_id_missing, order_id_unique,order_id_type)
# list of mills
print(df['mill'].unique())

mill                object
country_delivery    object
country             object
delivery_point      object
cost                 int64
order_id             int64
month_order         object
dtype: object
Missing values:  
 mill                0
country_delivery    0
country             0
delivery_point      0
cost                0
order_id            0
month_order         0
dtype: int64
mill 0 False int64
['Hobart' 'Darwin' 'Canberra' 'Adelaide' 'Perth' 'Brisbane' 'Melbourne']


In [4]:
# Find out which non-EU countries joined EU sanctions
# !pip install country_converter
import country_converter as coco
cc = coco.CountryConverter()
df['EU'] = cc.pandas_convert(series=df['country_delivery'], to='EU')
# Filter values of non-EU countries
df['country'].loc[df['EU'] == 'not found'].unique()

array(['Australia', 'China', 'United States', 'Albania', 'Norway',
       'Switzerland', 'Serbia', 'Turkey', 'Bosnia and Herzegovina',
       'Uzbekistan', 'Belarus'], dtype=object)

In [5]:
# add countries which joinded the sanctions
selected_countries = ['Albania', 'Bosnia and Herzegovina', 'Switzerland', 'Norway', 'United States']
df['sanctions'] = df['country'].isin(selected_countries) | (df['EU'] == 'EU')
df['sanctions'].fillna(value=False, inplace=True)
#add currency in separate column
df.loc[(df['country'] !='Australia'), 'currency'] = 'EUR'
df.loc[(df['country'] =='Australia'), 'currency'] = 'AUD'

## Mill capacity

Objective : define actual costs in order to assess the redistribution of the supply chain. 

Capacity is used as a proxy : each row per factory is considered as a unit of capacity. Therefore, we count the total deliveries per mill and calculate the percentage of sanctioned deliveries, per mill, out of total capacity

In [6]:
# Filter orders with sanctions
df_sanction = df[~df['sanctions'] == False]
sanctioned_delivery = df_sanction.groupby(['mill']).sanctions.count().reset_index()
total = df.groupby(['mill']).country.count().reset_index()
total.rename(columns = {'country': 'delivery'}, inplace = True)
# We count the cost of the sanctioned deliveries
cost_sanctioned = df_sanction.groupby(['mill']).cost.sum().reset_index()

In [7]:
# Assess impact of sanctions
performance = sanctioned_delivery.merge(total)
performance['percentage_sanctions'] = ((performance.sanctions/performance.delivery)*100).round(1)
performance = performance.merge(cost_sanctioned, how='inner')
performance.rename(columns = {'cosr': 'cost_sanctions_EUR'}, inplace = True)
performance['no_sanction'] = performance.delivery - performance.sanctions
#Checking if the numbers of our new table match with our df
print(df['mill'].value_counts().sum())
print(df['sanctions'].value_counts().sum())

12967
12967


In [13]:
# Save data into the excel file
# Create a Pandas Excel writer using ExcelWriter
with pd.ExcelWriter('./data/df_transportation_data.xlsx', engine='openpyxl') as writer:
    # Write the DataFrame to the Excel file
    df.to_excel(writer, sheet_name='data', index=False)
    # Create a new sheet and write 'no_sanction' data to it
    performance.to_excel(writer, sheet_name='performance', index=False)


## Distance function to assess transportation costs

Similar to an optimization function, the aim is to calculate the cost per distance and to use some vizualisation to assess the demand.  

SUPPLY: as a proxy, the maximum capacity (sum of number of deliveries) per mill has already been calculated previously. 

DEMAND: the sum of number of deliveries per destination (non sanctioned) is used as a proxy. 

COST: We use only our non sanctioned deliveries, the rest is irrelevant for our solution.

### Steps to accomplish
DEMAND: sum of deliveries per destination not sanctioned

COST: convert all in functional currency (RUB) at transaction date (historical rate), and calculate cost per km per mill-destination combination.

In [14]:
#we want to have a correct date formatting. Transform the month into a datetime format and correct the year (because it only contains the month, it sets default to year 1900)
df['day_order'] = pd.to_datetime(df['month_order'], format='%B')+pd.DateOffset(years= 121) 
#We now have our correct date, we can calculate historical XR with a set library
df.head()

,mill,country_delivery,country,delivery_point,cost,order_id,month_order,EU,sanctions,currency,day_order
0,Hobart,AU,Australia,Nowra,175000,10226209,November,not found,False,AUD,2021-11-01
1,Hobart,AU,Australia,Nowra,175000,10226230,November,not found,False,AUD,2021-11-01
2,Darwin,AU,Australia,Newcastle,171000,10206123,October,not found,False,AUD,2021-10-01
3,Darwin,AU,Australia,Newcastle,171000,10185874,September,not found,False,AUD,2021-09-01
4,Darwin,AU,Australia,North-west,170000,10176556,August,not found,False,AUD,2021-08-01


In [15]:
from forex_python.converter import CurrencyRates
# !pip install forex_python
from datetime import datetime
c = CurrencyRates()
# get rates for all unique dates, added tqdm progress bar to see progress
from tqdm import tqdm  
rates_dict = pd.DataFrame({date: CurrencyRates().get_rates('EUR', date_obj=pd.to_datetime(date, unit='D'))
              for date in tqdm(df['day_order'].unique())})

  0%|          | 0/12 [00:00<?, ?it/s]

100%|██████████| 12/12 [00:01<00:00,  6.10it/s]


In [16]:
# Using rates to set cost to 1/(AUD to currency_x rate), except when currency is AUD and when servicecost is 0, in those cases just use servicecost
rates_aud = rates_dict.T
rates_aud = rates_aud.loc[:,['AUD']]
rates_aud.index.name = 'day_order'
df = df.merge(rates_aud,on='day_order')
df.rename(columns = {'AUD': 'XR'}, inplace = True)
df.loc[(df['country'] =='Australia'),'XR'] = 1


In [17]:
# calculate cost in AUD and unify our cost column
df['cost_AUD'] = df.cost* df.XR
print(df.shape[1])
df.columns

13


Index(['mill', 'country_delivery', 'country', 'delivery_point', 'cost',
       'order_id', 'month_order', 'EU', 'sanctions', 'currency', 'day_order',
       'XR', 'cost_AUD'],
      dtype='object')

In [18]:
# Save data into the excel file
# Create a Pandas Excel writer using ExcelWriter
with pd.ExcelWriter('./data/df_transportation_data.xlsx', engine='openpyxl') as writer:
    # Write the DataFrame to the Excel file
    df.to_excel(writer, sheet_name='data', index=False)
    # Create a new sheet and write 'no_sanction' data to it
    performance.to_excel(writer, sheet_name='performance', index=False)

In [19]:
#we want to calculate the cost and demand per destination
df_no_sanction = df[~df['sanctions'] == True]
#remove whitespace, homogenize each delivery zone
df_no_sanction['country'] = df_no_sanction['country'].str.strip()
df_no_sanction['delivery_point'] = df_no_sanction['delivery_point'].str.strip()
#Generating a list to check individually, there are duplicates and formatting errors
list_zones = df_no_sanction['delivery_point'].unique()
list_inp = list_zones.tolist()
for item in list_inp: 
    print(item) 
#Create one column to get the combination country-city delivery point
df_no_sanction['delivery_zone'] = df_no_sanction['country'].astype(str) + "_" + df_no_sanction['delivery_point'].astype(str)

Nowra
East
Sydney
Ballarat
Geelong
Armadale
Toowoomba
Brisbane
Berwick
South
Melbourne
Centre
Mackay
North
Werribee
Caloundra
Blacktown
Canberra
Wodonga
Gomel
Newcastle
Albury
Manturovo
Dubbo
North-west
Sloboda
Cairns
Rockhampton
Shepparton
Bendigo
Tamworth
Melton
Bankstown
Epping
Bunbury
Lismore
Tashkent
Polochany
Rockingham
Launceston
Maitland
Kalgoorlie
Perth


In [20]:
demand = df_no_sanction.groupby(['country','mill','delivery_point','delivery_zone','cost_AUD'])['country_delivery'].count().reset_index()
demand.rename(columns = {'cost_AUD': 'demand_AUD'}, inplace = True)
demand.rename(columns = {'country_delivery': 'amount'}, inplace = True)
demand.drop(['amount'], axis=1)
#we take our total number per mill from our previous table
demand = demand.merge(total)
demand_per_combination  =  df_no_sanction.groupby(['delivery_zone'])['country_delivery'].count().reset_index()
demand_per_combination.rename(columns = {'country_delivery': 'total_country_zone'}, inplace = True)
demand = demand.merge(demand_per_combination)
demand


,country,mill,delivery_point,delivery_zone,demand_AUD,amount,delivery,total_country_zone
0,Australia,Adelaide,Armadale,Australia_Armadale,20000.0000,14,1396,84
1,Australia,Brisbane,Armadale,Australia_Armadale,74000.0000,1,1791,84
2,Australia,Brisbane,Armadale,Australia_Armadale,96000.0000,2,1791,84
3,Australia,Brisbane,Armadale,Australia_Armadale,100000.0000,50,1791,84
4,Australia,Brisbane,Armadale,Australia_Armadale,102000.0000,8,1791,84
...,...,...,...,...,...,...,...,...
780,Turkey,Perth,Centre,Turkey_Centre,3099.7200,1,1446,5
781,Turkey,Melbourne,South,Turkey_South,2518.9835,1,3892,4
782,Turkey,Melbourne,South,Turkey_South,4754.7000,1,3892,4
783,Turkey,Melbourne,South,Turkey_South,5547.1500,1,3892,4


In [21]:
# Calculate supply (total per mill) = done
# Converted all deliveries at historic rate (in RUB) = done
# Check and remove outliers per destination
import seaborn as sns
import warnings
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
%matplotlib inline
sns.set()
# Graphics in retina format are more sharp and legible
%config InlineBackend.figure_format = 'retina'

In [22]:
import geocoder
# !pip install geocoder
from pprint import pprint
key = '10134c1cacab408c8d41af91bfa80a77' # get api key from: https://opencagedata.com
mills = dict.fromkeys(demand['mill'])
cities = dict.fromkeys(demand['delivery_point'])
for name in mills:
    result = geocoder.opencage(name, key=key)
    mills[name] = result.latlng
for name in cities:
    result = geocoder.opencage(name, key=key)
    cities[name] = result.latlng

In [23]:
#Transform, rename, merge
mills_distance = pd.DataFrame.from_dict(mills, orient='index',columns=['lat_mill','long_mill'])
cities_distance = pd.DataFrame.from_dict(cities, orient='index',columns=['lat_city','long_city'])
mills_distance.index.name = 'mill'
cities_distance.index.name = 'delivery_point'
demand = demand.merge(mills_distance, left_on='mill', right_on='mill')
demand = demand.merge(cities_distance, left_on='delivery_point', right_on='delivery_point')

In [24]:
#Calculate distance between two points, calculate cost per km = proxy for cost minimization function
import h3
#!pip install h3
demand['distance'] = demand.apply(lambda row: h3.point_dist((row['lat_mill'], row['long_mill']), (row['lat_city'], row['long_city'])), axis=1)
demand['cost_per_km'] = demand.demand_AUD/demand.distance

In [25]:
#issue with inf type of float, which json doesn't recognise
demand[["lat_mill", "long_mill", "lat_city", "long_city", 'distance','cost_per_km']] = demand[["lat_mill", "long_mill", "lat_city", "long_city", 'distance','cost_per_km']].astype(str)
demand[["lat_mill", "long_mill", "lat_city", "long_city", 'distance','cost_per_km']] = demand[["lat_mill", "long_mill", "lat_city", "long_city", 'distance','cost_per_km']].astype(str)

In [26]:
#Some rows are very high in comparison to the distance, meaning the unit sold where much bigger than the km traveled
demand_sydney = demand[demand['delivery_zone'] == 'Australia_SYDNEY']
demand_sydney = demand_sydney.sort_values(by='cost_per_km', ascending = False)

In [27]:
# Save data into the excel file
# Create a Pandas Excel writer using ExcelWriter
with pd.ExcelWriter('./data/df_transportation_data.xlsx', engine='openpyxl') as writer:
    # Write the DataFrame to the Excel file
    df.to_excel(writer, sheet_name='data', index=False)
    performance.to_excel(writer, sheet_name='performance', index=False)
    demand.to_excel(writer,sheet_name='demand',index=False)

In [28]:
#issue with inf type of float, which json doesn't recognise
demand[["lat_mill", "long_mill", "lat_city", "long_city", 'distance','cost_per_km']] = demand[["lat_mill", "long_mill", "lat_city", "long_city", 'distance','cost_per_km']].astype(float)
demand[["lat_mill", "long_mill", "lat_city", "long_city", 'distance','cost_per_km']] = demand[["lat_mill", "long_mill", "lat_city", "long_city", 'distance','cost_per_km']].astype(float)

In [29]:
distance = pd.pivot_table(demand,values='distance',index=['mill','delivery_zone'], aggfunc='first')
cost = pd.pivot_table(demand,values='cost_per_km',index=['mill','delivery_zone'], aggfunc='first')

In [30]:
# Save data into the excel file
# Create a Pandas Excel writer using ExcelWriter
with pd.ExcelWriter('./data/df_transportation_data.xlsx', engine='openpyxl') as writer:
    # Write the DataFrame to the Excel file
    df.to_excel(writer, sheet_name='data', index=False)
    performance.to_excel(writer, sheet_name='performance', index=False)
    demand.to_excel(writer,sheet_name='demand',index=False)
    distance.to_excel(writer,sheet_name='distance',index=False)

# Visualisation part
The part below contains different scatterplots to visualise the data

In [32]:
#Let's visualise the production capacity on the map
demand_AU = demand[demand['country'] == 'Australia']
viz_AU = demand_AU.groupby(['mill','delivery','lat_mill','long_mill']).delivery_point.count().reset_index()
viz_AU

,mill,delivery,lat_mill,long_mill,delivery_point
0,Adelaide,1396,-34.928181,138.599931,77
1,Brisbane,1791,-27.468968,153.023499,93
2,Canberra,862,-35.297591,149.101268,55
3,Darwin,2540,-12.460440,130.841047,99
4,Hobart,1040,-42.882509,147.328123,75
5,Melbourne,3892,-37.814245,144.963173,128
6,Perth,1446,-31.955893,115.860585,108


In [33]:
import plotly.express as px
#!pip install plotly
color_scale = [(0, 'blue'), (1,'red')]

fig = px.scatter_mapbox(demand, 
                        lat="lat_city", 
                        lon="long_city", 
                        hover_name="delivery_point", 
                        hover_data=["mill", "cost_per_km"],
                        color="cost_per_km",
                        color_continuous_scale=color_scale,
                        size="cost_per_km",
                        zoom=8, 
                        height=700,
                        width=700)

fig.update_layout(mapbox_style="carto-positron")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [35]:
import plotly.express as px
color_scale = [(0, 'orange'), (1,'red')]

fig = px.scatter_mapbox(viz_AU, 
                        lat="lat_mill", 
                        lon="long_mill", 
                        hover_name="mill", 
                        hover_data=["mill", "delivery"],
                        color="delivery_point",
                        color_continuous_scale=color_scale,
                        size="delivery_point",
                        zoom=8, 
                        height=700,
                        width=700)

fig.update_layout(mapbox_style="carto-positron")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()